**Predicting Room Occupancy based on Temperature, Humidity,Light and CO2**

Dataset- https://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+

In this dataset, we have the following attributes

date time year-month-day hour:minute:second  
Temperature, in Celsius  
Relative Humidity, %  
Light, in L  
CO2, in ppm  
Humidity Ratio, Derived quantity from temperature and relative humidity, in kgwater-vapor/kg-air  
Occupancy, 0 or 1, 0 for not occupied, 1 for occupied status

In [267]:
import pandas as pd

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, GRU, Conv1D

np.random.seed(1)
tf.random.set_seed(1)


In [268]:
df = pd.read_csv('C:/Users/prath/Downloads/occupancy.csv',  parse_dates=True)

In [269]:
df.columns

Index(['date', 'Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio',
       'Occupancy'],
      dtype='object')

In [270]:
data=df.to_numpy() #converting to an array

In [271]:
data=data[:,1:] #extracting all the columns except date

In [272]:
data

array([[23.7, 26.272, 585.2, 749.2, 0.004764163, 1],
       [23.718, 26.29, 578.4, 760.4, 0.004772661, 1],
       [23.73, 26.23, 572.6666667, 769.6666667, 0.004765153, 1],
       ...,
       [24.33, 25.7, 817.0, 1125.8, 0.004840759, 1],
       [24.35666667, 25.7, 813.0, 1123.0, 0.004848559, 1],
       [24.40833333, 25.68166667, 798.0, 1124.0, 0.004860208, 1]],
      dtype=object)

In [273]:
data.shape

(2665, 6)

In [274]:
def create_sequences(data, seq_length):
    obs = []
    print(obs)    
    for i in range(len(data)-seq_length-1):
        obs.append(data[i:(i+seq_length)+1])
    return np.array(obs)

creating a sequence of 5 as we have 5 features

In [275]:
RNN_input = create_sequences(data, 5)
RNN_input

[]


array([[[23.7, 26.272, 585.2, 749.2, 0.004764163, 1],
        [23.718, 26.29, 578.4, 760.4, 0.004772661, 1],
        [23.73, 26.23, 572.6666667, 769.6666667, 0.004765153, 1],
        [23.7225, 26.125, 493.75, 774.75, 0.004743773, 1],
        [23.754, 26.2, 488.6, 779.0, 0.004766594, 1],
        [23.76, 26.26, 568.6666667, 790.0, 0.004779332, 1]],

       [[23.718, 26.29, 578.4, 760.4, 0.004772661, 1],
        [23.73, 26.23, 572.6666667, 769.6666667, 0.004765153, 1],
        [23.7225, 26.125, 493.75, 774.75, 0.004743773, 1],
        [23.754, 26.2, 488.6, 779.0, 0.004766594, 1],
        [23.76, 26.26, 568.6666667, 790.0, 0.004779332, 1],
        [23.73, 26.29, 536.3333333, 798.0, 0.004776136, 1]],

       [[23.73, 26.23, 572.6666667, 769.6666667, 0.004765153, 1],
        [23.7225, 26.125, 493.75, 774.75, 0.004743773, 1],
        [23.754, 26.2, 488.6, 779.0, 0.004766594, 1],
        [23.76, 26.26, 568.6666667, 790.0, 0.004779332, 1],
        [23.73, 26.29, 536.3333333, 798.0, 0.004776136,

In [276]:
RNN_input[0]

array([[23.7, 26.272, 585.2, 749.2, 0.004764163, 1],
       [23.718, 26.29, 578.4, 760.4, 0.004772661, 1],
       [23.73, 26.23, 572.6666667, 769.6666667, 0.004765153, 1],
       [23.7225, 26.125, 493.75, 774.75, 0.004743773, 1],
       [23.754, 26.2, 488.6, 779.0, 0.004766594, 1],
       [23.76, 26.26, 568.6666667, 790.0, 0.004779332, 1]], dtype=object)

In [277]:
y = RNN_input[:,-1,-1] # the last column of the last row is our target variable which is occupancy
y

array([1, 1, 1, ..., 1, 1, 1], dtype=object)

In [278]:
X = RNN_input[:,0:5,0:6] # getting all the rows except last one which will be used for training
X

array([[[23.7, 26.272, 585.2, 749.2, 0.004764163, 1],
        [23.718, 26.29, 578.4, 760.4, 0.004772661, 1],
        [23.73, 26.23, 572.6666667, 769.6666667, 0.004765153, 1],
        [23.7225, 26.125, 493.75, 774.75, 0.004743773, 1],
        [23.754, 26.2, 488.6, 779.0, 0.004766594, 1]],

       [[23.718, 26.29, 578.4, 760.4, 0.004772661, 1],
        [23.73, 26.23, 572.6666667, 769.6666667, 0.004765153, 1],
        [23.7225, 26.125, 493.75, 774.75, 0.004743773, 1],
        [23.754, 26.2, 488.6, 779.0, 0.004766594, 1],
        [23.76, 26.26, 568.6666667, 790.0, 0.004779332, 1]],

       [[23.73, 26.23, 572.6666667, 769.6666667, 0.004765153, 1],
        [23.7225, 26.125, 493.75, 774.75, 0.004743773, 1],
        [23.754, 26.2, 488.6, 779.0, 0.004766594, 1],
        [23.76, 26.26, 568.6666667, 790.0, 0.004779332, 1],
        [23.73, 26.29, 536.3333333, 798.0, 0.004776136, 1]],

       ...,

       [[24.218, 25.912, 805.0, 1152.4, 0.004848089, 1],
        [24.26, 25.89166667, 798.0, 1146.16

Train to Test Split

In [279]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [280]:
y_train

array([0, 0, 1, ..., 1, 1, 0], dtype=object)

In [281]:
y_train = y_train.reshape(-1,1)
y_train

array([[0],
       [0],
       [1],
       ...,
       [1],
       [1],
       [0]], dtype=object)

In [282]:
y_test = y_test.reshape(-1,1)
y_test

array([[0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
    

In [283]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2127, 5, 6)
(532, 5, 6)
(2127, 1)
(532, 1)


In [284]:
X_train

array([[[20.365, 22.5, 0.0, 436.5, 0.003321598, 0],
        [20.39, 22.5, 0.0, 434.0, 0.003326758, 0],
        [20.39, 22.5, 0.0, 435.25, 0.003326758, 0],
        [20.39, 22.5, 0.0, 432.75, 0.003326758, 0],
        [20.39, 22.5, 0.0, 432.0, 0.003326758, 0]],

       [[21.2, 27.6, 0.0, 758.5, 0.004296136, 0],
        [21.2, 27.575, 0.0, 749.75, 0.004292217, 0],
        [21.2, 27.6, 0.0, 749.0, 0.004296136, 0],
        [21.2, 27.54, 0.0, 747.8, 0.004286732, 0],
        [21.2, 27.5, 0.0, 744.0, 0.004280463, 0]],

       [[21.175, 25.04041667, 446.6666667, 751.5, 0.003889213, 1],
        [21.15, 24.9175, 449.3333333, 750.3333333, 0.003864032, 1],
        [21.18333333, 25.19833333, 454.6666667, 754.3333333, 0.00391591,
         1],
        [21.2, 25.34, 448.0, 765.75, 0.003942121, 1],
        [21.2, 25.3925, 454.75, 782.0, 0.00395034, 1]],

       ...,

       [[21.1, 24.89, 444.0, 740.8, 0.003847834, 1],
        [21.13333333, 24.82333333, 444.0, 747.3333333, 0.003845377, 0],
        [21.16

As Neural Networks are scale sensitive, it is better to rescale them for appropriate results

In [285]:
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = X_scaler.fit_transform(X_train.reshape(-1, 6))
X_test_scaled = X_scaler.transform(X_test.reshape(-1, 6))

In [286]:
X_train_scaled=X_train_scaled.reshape(-1, 5, 6)
X_test_scaled=X_test_scaled.reshape(-1, 5, 6)

**LSTM**

In [287]:
model = Sequential(
    [
        LSTM(64,return_sequences=True, input_shape=[5, 6]),
        LSTM(32, return_sequences=True),
        LSTM(32,return_sequences=False),
        Dense(1, activation='sigmoid')
    ]
)
model.summary()

model.compile(loss = 'binary_crossentropy', metrics = 'accuracy')


Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 5, 64)             18176     
                                                                 
 lstm_19 (LSTM)              (None, 5, 32)             12416     
                                                                 
 lstm_20 (LSTM)              (None, 32)                8320      
                                                                 
 dense_13 (Dense)            (None, 1)                 33        
                                                                 
Total params: 38,945
Trainable params: 38,945
Non-trainable params: 0
_________________________________________________________________


In [288]:
y_train = np.array(y_train, dtype=np.int32)

In [289]:
y_test = np.array(y_test, dtype=np.int32)

In [290]:
from tensorflow.keras.callbacks import EarlyStopping

import time
start_time = time.time()

early_stop = EarlyStopping(monitor='val_loss', min_delta = 0.0, patience=10, verbose=1, mode='min')

history = model.fit(
    X_train_scaled, 
    y_train, 
    epochs=100,
    validation_data=(X_test_scaled, y_test), 
    callbacks=[early_stop]
)

end_time = time.time()

Epoch 1/100
67/67 [==============================] - 4s 17ms/step - loss: 0.3428 - accuracy: 0.9450 - val_loss: 0.1249 - val_accuracy: 0.9699
Epoch 2/100
67/67 [==============================] - 1s 8ms/step - loss: 0.1053 - accuracy: 0.9732 - val_loss: 0.1190 - val_accuracy: 0.9699
Epoch 3/100
67/67 [==============================] - 1s 10ms/step - loss: 0.0988 - accuracy: 0.9756 - val_loss: 0.1168 - val_accuracy: 0.9718
Epoch 4/100
67/67 [==============================] - 1s 9ms/step - loss: 0.0921 - accuracy: 0.9751 - val_loss: 0.1148 - val_accuracy: 0.9699
Epoch 5/100
67/67 [==============================] - 1s 9ms/step - loss: 0.0860 - accuracy: 0.9770 - val_loss: 0.1077 - val_accuracy: 0.9718
Epoch 6/100
67/67 [==============================] - 1s 8ms/step - loss: 0.0808 - accuracy: 0.9774 - val_loss: 0.1012 - val_accuracy: 0.9756
Epoch 7/100
67/67 [==============================] - 1s 8ms/step - loss: 0.0764 - accuracy: 0.9784 - val_loss: 0.0998 - val_accuracy: 0.9793
Epoch 8/100

In [291]:
y_pred = model.predict(X_test_scaled)

17/17 [==============================] - 1s 3ms/step


Converting the results to have binary output

In [292]:
threshold = 0.5
y_pred_binary = (y_pred > threshold).astype(int)

In [293]:
results = pd.DataFrame()

results['actual'] = y_test.flatten()
results['predicted'] = y_pred_binary.flatten()
results['residual'] = results['actual'] - results['predicted']

results

,actual,predicted,residual
0,0,0,0
1,1,1,0
2,1,1,0
3,1,1,0
4,0,0,0
...,...,...,...
527,1,1,0
528,1,1,0
529,0,0,0
530,1,1,0


In [294]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred_binary)



0.9906015037593985

**GRU**

In [295]:
model = Sequential(
    [
        GRU(64,return_sequences=True, input_shape=[5, 6]),
        GRU(32, return_sequences=True),
        GRU(16,return_sequences=False),
        Dense(1, activation='sigmoid')
    ]
)
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_15 (GRU)                (None, 5, 64)             13824     
                                                                 
 gru_16 (GRU)                (None, 5, 32)             9408      
                                                                 
 gru_17 (GRU)                (None, 16)                2400      
                                                                 
 dense_14 (Dense)            (None, 1)                 17        
                                                                 
Total params: 25,649
Trainable params: 25,649
Non-trainable params: 0
_________________________________________________________________


In [296]:
model.compile(
    loss="binary_crossentropy", 
    optimizer='sgd', 
    metrics='accuracy'
)

In [297]:
from tensorflow.keras.callbacks import EarlyStopping

import time
start_time = time.time()

early_stop = EarlyStopping(monitor='val_loss', min_delta = 0.0, patience=10, verbose=1, mode='min')

history = model.fit(
    X_train_scaled, 
    y_train, 
    epochs=100,
    validation_data=(X_test_scaled, y_test), 
    callbacks=[early_stop]
)

end_time = time.time()

Epoch 1/100
67/67 [==============================] - 4s 15ms/step - loss: 0.6418 - accuracy: 0.6380 - val_loss: 0.5794 - val_accuracy: 0.7162
Epoch 2/100
67/67 [==============================] - 0s 7ms/step - loss: 0.5143 - accuracy: 0.8655 - val_loss: 0.4476 - val_accuracy: 0.9586
Epoch 3/100
67/67 [==============================] - 0s 7ms/step - loss: 0.3756 - accuracy: 0.9751 - val_loss: 0.3104 - val_accuracy: 0.9774
Epoch 4/100
67/67 [==============================] - 0s 7ms/step - loss: 0.2526 - accuracy: 0.9751 - val_loss: 0.2098 - val_accuracy: 0.9774
Epoch 5/100
67/67 [==============================] - 1s 8ms/step - loss: 0.1749 - accuracy: 0.9779 - val_loss: 0.1544 - val_accuracy: 0.9774
Epoch 6/100
67/67 [==============================] - 0s 7ms/step - loss: 0.1336 - accuracy: 0.9774 - val_loss: 0.1255 - val_accuracy: 0.9793
Epoch 7/100
67/67 [==============================] - 0s 7ms/step - loss: 0.1120 - accuracy: 0.9774 - val_loss: 0.1097 - val_accuracy: 0.9793
Epoch 8/100


In [298]:
y_pred = model.predict(X_test_scaled)

17/17 [==============================] - 1s 3ms/step


In [299]:
threshold = 0.5
y_pred_binary = (y_pred > threshold).astype(int)

In [300]:
results = pd.DataFrame()

results['actual'] = y_test.flatten()
results['predicted'] = y_pred_binary.flatten()
results['residual'] = results['actual'] - results['predicted']

results

,actual,predicted,residual
0,0,0,0
1,1,1,0
2,1,1,0
3,1,1,0
4,0,0,0
...,...,...,...
527,1,1,0
528,1,1,0
529,0,0,0
530,1,1,0


In [301]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred_binary)

0.9830827067669173

Out of the two recurrent models I have fit on the data i.e., LSTM and GRU. LSTM fit the data better than GRU with an accuracy of 0.99. There is not much difference in the performance. Even GRU has an accuracy of 0.98